In [1]:
import numpy
import pandas as pd
import io
from itertools import permutations

In [19]:
def read_file_init_table(fname):
    tag_count = {}
    tag_count['<start>'] = 0
    word_tag = {}
    tag_trans = {}
    with open(fname) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    idx_line = 0
    is_first_word = 0
    
    while idx_line < len(content):
        prev_tag = '<start>'
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                if content_part[1] in tag_count:
                    tag_count[content_part[1]] += 1
                else:
                    tag_count[content_part[1]] = 1
                    
#                 current_word_tag = content_part[0]+','+content_part[1]
                if (content_part[0],content_part[1]) in word_tag:
                    word_tag[(content_part[0],content_part[1])] += 1
                else:    
                    word_tag[(content_part[0],content_part[1])] = 1
                    
                if is_first_word == 1:
                    current_tag_trans = ('<start>',content_part[1])
                    is_first_word = 0
                else:
                    current_tag_trans = (prev_tag,content_part[1])
                    
                if current_tag_trans in tag_trans:
                    tag_trans[current_tag_trans] += 1
                else:
                    tag_trans[current_tag_trans] = 1                    
                prev_tag = content_part[1]   
                
            else:
                tag_count['<start>'] += 1
                is_first_word = 1
            idx_line = idx_line + 1

        idx_line = idx_line+1       
    return tag_count, word_tag, tag_trans

def read_test(fname):
    with open(fname) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    idx_line = 0
    tag = []
    kalimatt = []
    while idx_line < len(content):
        kalimat = []
        tagg = []
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                kalimat.append(content_part[0])
                tagg.append(content_part[1])
            idx_line = idx_line + 1
        idx_line = idx_line+1
        kalimatt.append(kalimat)
        tag.append(tagg)
    return tag,kalimatt

def create_trans_prob_table(tag_trans, tag_count):
    trans_prob = {}
    for tag1 in tag_count.keys():
        for tag2 in tag_count.keys():
            #print('tag1 = ')
            #print(tag1)
            trans_idx = (tag1,tag2)
            #print('trans_idx = ')
            #print(trans_idx)
            if trans_idx in tag_trans:
                #print(trans_idx)
                trans_prob[trans_idx] = tag_trans[trans_idx]/tag_count[tag1]
    return trans_prob

def create_emission_prob_table(word_tag, tag_count):
    emission_prob = {}
    keys = word_tag.keys()
    for key in keys:
        current_word = key[0]
        current_tag = key[1]
        emission_prob[(current_word,current_tag)] = word_tag[(current_word,current_tag)]/tag_count[current_tag]    
    return emission_prob

def findTag(emission_prob,word):
    x = []
    for i in emission_prob:
        if i[0] == word:
            x.append(i[1])
    return x

def findAllTag(emission_prob,sentence):
    x = []
    for i in sentence:
        x = list(set(x+findTag(emission_prob,i)))
    return x

In [20]:
tag_count, word_tag, tag_trans = read_file_init_table("Indonesian_Manually_Tagged_Corpus_ID-Train.tsv")
# print(tag_count)
# print(word_tag)
# print(tag_trans)

In [21]:
modelTag = {}
modelKata = []
for i in word_tag:
    if i[1] in modelTag:
        modelTag[i[1]] += 1
    else :
        modelTag[i[1]] = 1
    if i[0] not in modelKata:
        modelKata.append(i[0])
tagTerbanyak = max(modelTag, key=lambda i: modelTag[i])

In [27]:
trans_prob

{('<start>', 'CC'): 0.0854368932038835,
 ('<start>', 'CD'): 0.02524271844660194,
 ('<start>', 'DT'): 0.003883495145631068,
 ('<start>', 'FW'): 0.002912621359223301,
 ('<start>', 'IN'): 0.1766990291262136,
 ('<start>', 'JJ'): 0.003883495145631068,
 ('<start>', 'MD'): 0.000970873786407767,
 ('<start>', 'NEG'): 0.000970873786407767,
 ('<start>', 'NN'): 0.3116504854368932,
 ('<start>', 'NND'): 0.0048543689320388345,
 ('<start>', 'NNP'): 0.16407766990291262,
 ('<start>', 'PR'): 0.01650485436893204,
 ('<start>', 'PRP'): 0.06990291262135923,
 ('<start>', 'RB'): 0.01262135922330097,
 ('<start>', 'SC'): 0.062135922330097085,
 ('<start>', 'UH'): 0.002912621359223301,
 ('<start>', 'VB'): 0.02912621359223301,
 ('<start>', 'WH'): 0.000970873786407767,
 ('<start>', 'Z'): 0.02524271844660194,
 ('CC', 'CC'): 0.011097410604192354,
 ('CC', 'CD'): 0.06535141800246609,
 ('CC', 'DT'): 0.0012330456226880395,
 ('CC', 'FW'): 0.02342786683107275,
 ('CC', 'IN'): 0.0468557336621455,
 ('CC', 'JJ'): 0.043156596794

In [22]:
trans_prob = create_trans_prob_table(tag_trans, tag_count)
emission_prob = create_emission_prob_table(word_tag, tag_count)

In [41]:
def viterbi(trans_prob, emission_prob, tag_count, sentence_words,modelKata,tagTerbanyak):
    #initialization
    for i in sentence_words:
        if i not in modelKata:
            modelKata.append(i)
            emission_prob[(i,tagTerbanyak)] = 1/tag_count[tagTerbanyak]
            tag
    allTag = findAllTag(emission_prob,sentence_words)
    #membuat mini transition table untuk suatu kalimat
    Ttable = {}
    for i in allTag:
        if ('<start>',i) in trans_prob:
            Ttable[('<start>',i)] = trans_prob[('<start>',i)]
        else:
            Ttable[('<start>',i)] = 0
        
    for tag1 in allTag:
        for tag2 in allTag:
            if ((tag1,tag2) in trans_prob):
                Ttable[(tag1,tag2)] = trans_prob[(tag1,tag2)]
            else:
                Ttable[(tag1,tag2)] = 0
    #membuat mini etable untuk kasus kata tersebut saja            
    Etable = {}
    for word in sentence_words:
        for tag in allTag:
            if ((word,tag) in emission_prob):
                Etable[(word,tag)] = emission_prob[(word,tag)]
            else:
                Etable[(word,tag)] = 0
                
    #melakukan algoritma viterbi dengan cara forward             
    hasil = {}
    track = {}
    for i in range (len(sentence_words)):
        word = sentence_words[i]
        if i == 0:
            start = '<start>'
            for tag in allTag:
                hasil[(word,tag)] = Etable[(word,tag)]*Ttable[(start,tag)]
                track[(word,tag)] = tag
#                 print (start,tag,hasil)
        else:
            for tag1 in allTag:
                temp = {}
                for tag2 in allTag:
#                     print (hasil[(sentence_words[i-1],tag2)],Ttable[(tag2,tag1)],tag2,tag1)
                    temp[(sentence_words[i-1],tag2)] = hasil[(sentence_words[i-1],tag2)]*Ttable[(tag2,tag1)]
#                 print (word,tag1,temp,"\n")
                maks = max(temp, key=lambda i: temp[i])
                hasil[(word,tag1)] = temp[maks] * Etable[(word,tag1)]
                track[(word,tag1)] = maks
    
    #viterbi backward
#     print (hasil,track)
    lastWord = sentence_words[len(sentence_words)-1]
    tagg = allTag[0]
    posTag = [tagg]
    maksimum = hasil[(lastWord,tagg)]
    for i in range(1,len(allTag)):
        if hasil[(lastWord,allTag[i])] > maksimum:
            maksimum = hasil[(lastWord,allTag[i])]
            posTag = [allTag[i]]
    for i in range (len(sentence_words)-2,-1,-1):
#         print ((sentence_words[i+1],posTag[0]))
        posTag.insert(0,track[(sentence_words[i+1],posTag[0])][1])
    return posTag

In [42]:
tagTest,kalimatTest = read_test("Indonesian_Manually_Tagged_Corpus_ID-Test.tsv")
predTest = []
for kalimat in kalimatTest:
    predTest.append(viterbi(trans_prob, emission_prob, tag_count, kalimat, modelKata,tagTerbanyak))

In [43]:
benar = 0
total = 0
for i in range(len(predTest)):
    for j in range(len(predTest[i])):
        if predTest[i][j] == tagTest[i][j]:
            benar+= 1
        total += 1
print (benar/total * 100)

83.96793587174348


In [44]:
total

499